# SetUp

In [6]:
# Import necessary libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import argparse
from collections import Counter

# For better visualization aesthetics
%matplotlib inline
plt.style.use('seaborn')


ImportError: cannot import name 'broadcast_to' from 'numpy.lib.stride_tricks' (/home/meos/Documents/MapReduceNeuralNetwork/env/lib/python3.9/site-packages/numpy/lib/stride_tricks.py)